In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
#insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/Users/Zacharias/Uni/ETH/FS2019/THESIS/code/ProGraML')
!pwd

/Users/Zacharias/Uni/ETH/FS2019/THESIS/code/ProGraML/deeplearning/ml4pl/poj104


In [3]:
ds_base = Path('classifyapp_data/ir_val')
print(ds_base.name)

ir_val


In [4]:
def get_dataloader(ds_base, batch_size=4):
    datalist = []
    out_base = ds_base.parent / (ds_base.name + '_pygeom')
    print(f"=== DATASET {ds_base}: getting dataloader")

    folders = [x for x in out_base.glob('*') if x.is_dir()]
    for folder in tqdm.tqdm(folders):
        # print(f"=== Opening Folder {str(folder)} ===")
        label = folder.name
        for k, file in enumerate(folder.glob('*.data.p')):
            #print(f"{k} - Processing {str(file)} ...")
            with open(file, 'rb') as f:
                data = pickle.load(f)
            datalist.append(data)
    print(f" * COMPLETED * === DATASET {ds_base}: returning dataloader")
    return DataLoader(datalist, batch_size=batch_size, shuffle=True)


In [5]:
loader = get_dataloader(ds_base)

  0%|          | 0/104 [00:00<?, ?it/s]

=== DATASET classifyapp_data/ir_val: getting dataloader


100%|██████████| 104/104 [00:09<00:00, 10.75it/s]

 * COMPLETED * === DATASET classifyapp_data/ir_val: returning dataloader


In [6]:
# Inspect batches

for batch in loader:
    edge_lists = []
    for i in range(3):
        mask = batch.edge_attr.squeeze() == i
        edge_list = batch.edge_index[:, mask].t()
        print(edge_list.size())
        edge_lists.append(edge_list)
    
    edge_positions = [
        torch.zeros_like(edge_lists[i])[:, 1]
        for i in range(3)
      ]
    break

torch.Size([405, 2])
torch.Size([757, 2])
torch.Size([37, 2])


In [7]:

from deeplearning.ml4pl.models.ggnn.ggnn_modules import GGNNModel
from deeplearning.ml4pl.models.ggnn.pygeom_ggnn_config import GGNNConfig

config = GGNNConfig(num_classes=104, has_graph_labels=True)
model = GGNNModel(config)

# set some global config values
dev = (
      torch.device("cuda")
      if torch.cuda.is_available()
      else torch.device("cpu")
    )

for batch in loader:
    edge_lists = []
    for i in range(3):
        mask = batch.edge_attr.squeeze() == i
        edge_list = batch.edge_index[:, mask].t()
        edge_lists.append(edge_list)
    
    edge_positions = [
        torch.zeros_like(edge_lists[i])[:,1]
        for i in range(3)
      ]
    
    model(batch.x.squeeze(), torch.zeros_like(batch.x), batch.y, edge_lists, edge_positions)

Initializing with random embeddings


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [7]:
# set some global config values
dev = (
      torch.device("cuda")
      if torch.cuda.is_available()
      else torch.device("cpu")
    )

In [ ]:
%debug
for batch in loader:
    edge_lists = []
    for i in range(3):
        mask = batch.edge_attr.squeeze() == i
        edge_list = batch.edge_index[:, mask].t()
        print(edge_list.size())
        edge_lists.append(edge_list)
    
    edge_positions = [
        torch.zeros_like(edge_lists[i])[1,:]
        for i in range(3)
      ]
    
    model(batch.x, torch.zeros_like(batch.x), batch.y, edge_lists, edge_positions)

> /Users/Zacharias/Uni/ETH/FS2019/THESIS/code/ProGraML/deeplearning/ml4pl/models/ggnn/ggnn_modules.py(426)forward()
    424       divisor = bincount.float()
    425       divisor[bincount == 0] = 1.0  # avoid div by zero for lonely nodes
--> 426       messages_by_targets /= divisor.unsqueeze_(1) + SMALL_NUMBER
    427     return messages_by_targets
    428 



In [ ]:
for batch in loader:
    break

In [ ]:
mask = batch.edge_attr.squeeze() == 0

In [ ]:
batch.edge_index[:, mask].size()

In [8]:
batch.x.size()

torch.Size([849, 1])

In [15]:
a = torch.ones([970,1,200])
a.size()

torch.Size([970, 1, 200])

In [16]:
b = torch.ones([970,1,1])
b.size()

torch.Size([970, 1, 1])

In [18]:
(a / b).size()

torch.Size([970, 1, 200])

In [19]:
a /= b

In [20]:
a.size()

torch.Size([970, 1, 200])

In [8]:
loader

In [9]:
dir(loader)

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_dataset_kind',
 '_index_sampler',
 'batch_sampler',
 'batch_size',
 'collate_fn',
 'dataset',
 'drop_last',
 'multiprocessing_context',
 'num_workers',
 'pin_memory',
 'sampler',
 'timeout',
 'worker_init_fn']

In [10]:
loader.batch_size

4